In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display, Image
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split



## Read in new data set
---

In [ ]:
df = pd.read_csv("data_cleaned.csv")
df

### Preperation for model
---

* ### Replace nan values with mean of column

In [ ]:
df.isna().sum()

In [ ]:
df = df.fillna(df.mean())

In [ ]:
df.isna().sum()

* ### Seperate target and features

In [ ]:
x = df.drop(['id','time', 'mood'],axis=1) #Exclude time for now, but try including it later!
y = df['mood']

* ### Normalise data
    "Neural networks' weights are updated by an amount proportional to the partial derivative of the loss function with respect to each weight. If features have very different scales, then the updates made to the weights will also differ in scale, making the learning process unstable. This is particularly critical for deep learning where different layers might have different sensitivities to the scale of input data."


In [ ]:

#Normalising the data such that each column has a mean of 0 and a standard deviation of 1 (i.e a Z score)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


* ### Reshape data for RNN

In [ ]:
time_steps = 1
x_scaled = np.array([x_scaled[i - time_steps:i, :] for i in range(time_steps, len(x_scaled) + 1)])
y[time_steps:] #Intuition: We are using day 1 to predict day 2, hence the y vector must start from day 2, as this is the predicted target using day 1's data 

x_scaled.shape

#Visualisation of transformation of x_scaled
#---------------------------------

#[[1,2,3,4,5,6,7,8,9,10,11]] #Looks like this for 1414 rows, where each row contins an array with one day worth of data


### Split data into training, testing and validation
---

In [ ]:
# For now we will allocate 20% of the data for testing (allocated randomly with seed of 42)
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

## Define and train model
---

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])), #First layer is a LSTM with 50 neurons
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse') 

In [ ]:
#Train model and randomly select 10% of the data for validation
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
mse_training = history.history['loss']
mse_validation = history.history['val_loss']
epochs = range(1, len(mse_training) + 1)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(epochs, mse_training, 'bo', label='Training loss')  
plt.plot(epochs, mse_validation, 'b', label='Validation loss')   
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Theory
---

* ###  Vanilla RNN

In [ ]:
display(Image(filename='RNN.png'))

* ###  LSTM RNN
    * An imporved RNN that fixes the exploding gradient and disappearing gradient problem

In [ ]:
display(Image(filename='LSTM RNN.png'))